# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - dwalton76's NxNxN Cube Solver</div>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [INFO](#0)
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
  * [FUNCTIONS FROM CRODOC](#3.1)
* [SOLVE](#5)
  * [SCORE](#5.1)
  * [SUBMISSION](#5.2)

<a id="0"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >INFO<br><div> 
    
Some research online led me to GitHub user [dwalton76's rubiks-cube-NxNxN-solver repo](https://github.com/dwalton76/rubiks-cube-NxNxN-solver). I have implemented the NxNxN cube solver below. Note that the solver will skip cubes with a solution state like "N1;N2;N3;..." or "A;B;A;...". The trick I used on the 3x3x3 cubes causes issues when N>=4 (more to come).
    
Special thanks to @wrrosa's Kociemba notebook. This notebook contains modifications of code found [here](https://www.kaggle.com/code/wrrosa/santa-2023-kociemba-s-two-phase-algo-1-116-550)

<a id="1"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >IMPORTS<br><div> 
    


In [11]:
%%bash
git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
cd rubiks-cube-NxNxN-solver
make init

fatal: destination path 'rubiks-cube-NxNxN-solver' already exists and is not an empty directory.


rm -rf build dist venv rubikscubennnsolver.egg-info cache ida_search ida_search_via_graph my-pt-states.txt
find . -name __pycache__ | xargs rm -rf
gcc -O3 -o ida_search_via_graph rubikscubennnsolver/ida_search_core.c rubikscubennnsolver/rotate_xxx.c rubikscubennnsolver/ida_search_666.c rubikscubennnsolver/ida_search_777.c rubikscubennnsolver/ida_search_via_graph.c -lm
python3 -m venv venv
  Using cached pip-23.3.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Obtaining file:///kaggle/working/rubiks-cube-NxNxN-solver (from -r requirements.dev.txt (line 10))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements 

In [12]:
%%bash
cd ..
git clone https://github.com/dwalton76/kociemba.git
cd kociemba/kociemba/ckociemba/
make
sudo make install

fatal: destination path 'kociemba' already exists and is not an empty directory.


mkdir -p bin
gcc -std=c99 -O3 -D_XOPEN_SOURCE=700 coordcube.c cubiecube.c facecube.c search.c -Iinclude solve.c -o bin/kociemba
cp bin/kociemba /usr/local/bin/


In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

os.chdir('../rubiks-cube-NxNxN-solver')
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  F:\AcroWorks\work\kaggle\rubiks-cube-NxNxN-solver


<a id="2"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 
    
It is becoming difficult to track down the source of some solutions due to all of the ensemble notebooks in use. I've attempted to locate the sources for the files that give the best solution. The Visualize Using Pythreejs appears to use some solutions from a previous version of this notebook: https://www.kaggle.com/code/zaburo/iterative-replacement-of-k-successive-moves
    
Ensemble sources:

-https://www.kaggle.com/code/yunsuxiaozi/k-step-optimal-solution-santa
-https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards
-https://www.kaggle.com/code/jazivxt/visualize-using-pythreejs
-https://www.kaggle.com/code/seanbearden/kociemba-w-greedy-all-3x3x3-ensemble

In [3]:
# sol1 = pd.read_csv('/kaggle/input/k-step-optimal-solution-santa/submission.csv')
# sol2 = pd.read_csv('/kaggle/input/bidirectional-brute-force-w-wildcards/submission.csv')
# sol3 = pd.read_csv('/kaggle/input/visualize-using-pythreejs/submission.csv')
# sol4 = pd.read_csv('/kaggle/input/kociemba-w-greedy-all-3x3x3-ensemble/submission.csv')


# ens = pd.DataFrame({
#     'solution1': sol1['moves'],
#     'solution2': sol2['moves'],
#     'solution3': sol3['moves'],
#     'solution4': sol4['moves'],
# })

# ens['ens_solution'] = ens.apply(lambda moves: min(moves, key=lambda x: len(x.split('.'))), axis=1)
# ensemble_score = ens['ens_solution'].str.split('.').apply(lambda x: len(x)).sum()

p = './'
puzzles = pd.read_csv(p + 'puzzles.csv', index_col='id')
puzzle_info = pd.read_csv(p + 'puzzle_info.csv', index_col='puzzle_type')
submission = pd.read_csv(p + 'submission.csv', index_col='id')

# submission['moves'] = ens['ens_solution']

<a id="3"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [4]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}
def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


In [5]:
def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

<a id="3.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">FUNCTIONS FROM CRODOC<br><div>

* https://www.kaggle.com/code/crodoc/1-187-898-greedy-baseline-improvement

In [6]:
allowed_moves = {}

for puzzle_type, row in puzzle_info.iterrows():
    allowed_moves[puzzle_type] = eval(row['allowed_moves'])

def move_state(state, move, moves_pool):
    
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state

def solve_puzzle(current_puzzle, moves_str):
    # paths.loc[paths['id'] == puzzle_id]
    # todo: assumes index aligned with puzzle id. Need to update.
#     current_puzzle = path.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
#     moves = submission_df.loc[puzzle_id]['moves'].split('.')
    moves = moves_str.split('.')
    moves_pool = allowed_moves[puzzle_type]
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # ignore last state
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx]
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

<a id="5"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 
    
Solver will skip cubes with a solution state like "N1;N2;N3;..." or "A;B;A;...".

In [8]:
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(281, 284):
            continue
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = move_translation(dim)

        init_state = row['initial_state']
        sol_state = row['solution_state']

        # Only attempt to solve cubes with traditional solutions. 
        # That is, cubes that have a solution state where each face 
        # filled with equivilent letters/colors.
        if sol_state[:2*dim**2-1] == ';'.join(['A']*dim**2):
            state = state2ubl(init_state)
            print(f'Starting {id}')
            print(state)

Starting 281
FDRFLDUDDRFUDUBRRFLLRBFBLBRBFDRURUFUULLULUBRRRDRDULLULBBDBFDLBFBFBRLDUULBDLDRUURLFUDFLBLLDRFUFBUULBUFUBFFDDRBRULBRRDLLFUDRBDUURDFBBFBDBBBFDRRLFDBURDDLBDLUFFFFBLFUURUDBFRFRDRFLRRLBDBBFDULRUDFFRBBRDRRDBDBDLLFRDFDBUUDUFBULLRUBFFFFDUURURDLFURBBLFBBRLUFFDRBRRFFFLRDFDRDBBFFDBLRRLUBFDUUFDFLRDBRRRBFRURDBFUUDLLLBBUFRRUDUUULURUURBDFBBRDRDBRFBFRDDRDUFBUDFRRFDRLLDFBRUUUUDFRFRDUFDLRFUFUUFRBDDUBLDFDLDBDLRURDULLBUUBRDULULFUFLUBFDLRRURUFUUFRBURFDRRLBLRDDLURRLRRLRRBBLLLDRFUFFLDRBUUFUUDRUUBRUUBDLDBLFFDLFLULLLFUBULLFLUDURFFRFFFFFDDULDUDBBBDFDRBFBULFLFBBBRRBDURULFFLRRFDUFFLRLFULLULBBRFDUDRBUDLULDULLDUDBBRDLRFRURLFBLRDLLDULUBBDDUDLRFDBDLFURRURDBBRDFFUFFRDDFURULFFURBRRDDUFULBUDDBBLLDRRBDBDUBFLFBRRDFBLLLRBRBUBDFFBBRLUDRRULRLUBDULBULBRLLRUURUFDFBLLFUDDFLBLFRRRDDRFULLUFFLDDFRUFLLURLDLRUUFUBFUDDUBBFDFBRRULULDDDFBBRFUDRLBDFRLBDBBFRBRFBUBFUFUDLLUBURFUUBBUDFULBUURFULLLURULUUBFRBBURLRBRFDDLLFUBDLULRRBLDLRULUDBDBULLRDFLRDLDDRBRBFDLRUUBDUULDFDFBLFUBFFDUUDRDRFFUUFBLUDBBUUURFRLDFUUBBDRUUBLUDFBUURRRDUBRRUDU

In [11]:
sol = "16Fw U' 16Lw 16Uw' 16Bw 16Rw 16Uw 16Fw2 L2 16Uw' 15Uw2 16Fw2 16Uw2 F 16Rw 16Uw2 15Rw2 U' 16Fw2 16Dw2 L2 15Fw 16Lw2 15Fw2 15Rw2 15Uw' 15Rw' 15Dw' 15Lw2 15Rw' B 15Dw R2 15Fw' R2 14Bw 16Lw' 16Rw 14Fw 16Dw2 16Lw U' 14Lw2 14Uw 16Fw2 U' 14Uw' 15Rw 14Uw 15Lw' R 15Dw2 14Uw' R 14Dw L 15Uw2 U' 14Fw' 14Dw' 14Lw' 14Bw' F' 16Rw2 D' 16Fw2 16Rw2 13Dw' 13Bw 16Lw 13Fw D U 16Rw2 13Uw2 15Uw2 13Bw' 15Rw' F 15Lw' R' 15Dw2 13Fw 13Dw B' 15Rw2 14Fw 14Uw2 14Lw' D2 F 14Lw' 14Rw2 14Bw' 14Rw2 14Fw2 14Dw2 14Rw D2 13Uw 14Rw' 14Uw2 14Rw' F2 13Rw' B 14Dw2 13Fw 13Rw' D' 13Bw' 13Fw' 13Uw' 16Fw2 16Lw 16Rw' 12Bw' D 16Bw2 16Lw 16Rw' B' 16Dw2 12Fw2 12Dw' 16Lw 15Lw R' 15Dw2 15Uw2 15Lw2 U' 15Fw2 12Uw' 15Lw' 12Bw F 15Lw L2 12Uw2 14Rw' 14Dw2 12Bw' 14Rw F' 14Dw2 12Lw 12Uw' 12Fw 14Lw2 13Bw' 13Uw2 13Rw' D' 13Lw2 13Bw2 D' R' 13Fw' 12Bw 13Dw2 12Lw' 13Bw2 13Rw2 F 13Lw 12Fw' 13Rw' F 12Uw 13Rw' L' 12Bw' 12Uw L2 12Dw2 12Bw 12Dw' 16Fw2 16Lw2 16Fw2 16Rw2 11Fw' 16Dw2 16Uw2 R' 16Bw2 11Uw' 16Rw 15Fw2 U2 11Bw 15Fw2 15Rw' 15Bw2 15Rw D2 15Rw' B2 11Fw' 15Rw2 11Uw2 F2 14Lw2 14Dw2 L2 14Fw2 11Dw F' 14Lw2 11Fw 14Lw' 14Rw' 11Uw2 11Fw' 13Lw' 13Uw2 R' 13Bw2 13Dw2 11Uw' 13Lw 11Bw L B' 13Dw2 12Fw' 12Rw 12Fw 12Uw2 12Fw L 12Fw' 11Uw B' 12Lw' 12Fw2 U' 12Uw2 11Bw' 12Lw 11Bw' 12Rw 11Uw B2 12Lw2 U' 11Fw' 11Rw' 11Bw' 11Lw2 11Dw' 16Uw2 16Fw2 16Rw 10Dw' 10Lw' 16Bw2 10Dw' 16Rw 10Dw' F' 16Dw2 15Uw2 B' 15Dw2 10Fw' 15Lw B 10Dw' 15Fw2 U2 10Lw D' 15Bw2 F' 14Uw2 14Bw2 U 14Lw' 10Bw' 10Dw' 14Rw 14Fw2 14Uw2 14Fw2 L 14Fw2 13Uw2 13Fw2 10Uw 13Bw2 13Lw' D' 13Fw2 13Uw2 L2 10Fw' 13Dw2 10Bw' 10Uw 13Lw' 10Uw' 12Rw' 12Fw2 10Dw2 F 12Rw' 12Dw2 10Fw 10Uw' F2 12Rw' U' 11Bw 11Rw' 11Uw2 11Rw' 11Uw2 11Fw2 R 11Bw' 11Rw 11Fw2 D' 11Rw2 11Fw2 11Lw' 11Fw2 10Rw2 F' 11Dw2 10Rw 10Fw' 11Rw' U' 10Fw L' 10Dw' 10Bw 10Rw' 10Dw' R' 16Uw2 16Lw2 9Dw 16Lw' 16Bw2 9Lw 16Lw' 16Dw2 9Lw2 F 16Lw' U2 15Lw' 15Bw2 9Fw' 9Uw' 15Fw2 R2 15Uw2 9Bw 15Lw 15Fw2 9Lw 15Uw2 15Lw 14Rw2 U2 14Bw2 9Fw' 14Lw2 9Dw2 9Rw' 14Bw2 9Lw U 14Lw2 13Bw2 13Uw2 13Lw R2 13Dw2 13Uw2 13Lw2 9Uw' 9Lw' 9Fw' 13Uw2 9Uw' B' 12Dw2 12Lw' 12Fw2 12Uw2 9Fw' L 12Rw 12Uw2 F U2 12Rw 11Uw2 9Bw2 9Uw 11Lw' 11Dw2 R' 11Bw2 11Uw2 9Rw2 9Bw' 9Dw 11Rw 10Uw2 10Fw 10Uw' 10Lw' U2 10Uw 10Rw 10Dw2 R 10Bw' U 10Fw2 10Lw2 9Fw 10Rw2 10Fw2 10Dw2 9Fw' 10Uw2 9Bw2 10Rw 9Bw D 10Rw' 9Uw F 9Fw2 9Lw 9Bw' 9Uw 9Fw' F' 16Fw2 16Dw2 16Rw' 8Uw F' 16Dw2 8Bw 16Bw2 8Rw 8Uw' 16Bw2 15Fw2 15Uw2 8Bw' 15Lw2 8Bw' D2 15Lw 15Dw2 8Fw 8Rw2 8Uw' 15Fw2 14Uw2 14Fw2 D2 14Rw' 14Uw2 14Lw2 14Fw2 R2 F 14Lw2 D 8Dw 14Rw 8Uw' 13Uw2 13Fw2 13Lw 13Rw' 8Dw' F2 13Lw2 R 8Uw' B2 13Lw' 8Fw2 R' 12Uw2 12Rw2 12Bw2 12Fw2 R' 8Bw' 12Lw' 8Uw' 8Rw2 12Bw2 8Uw2 11Rw2 D' 11Lw2 11Rw' 11Fw2 8Rw2 D 11Rw' 8Dw' 8Fw 11Lw 8Uw 8Bw 10Fw2 10Lw' 10Bw2 10Lw2 8Fw 8Rw2 10Rw' 8Fw U2 10Rw2 9Uw 9Lw 9Rw' 9Dw2 R2 9Uw L' 9Rw' 9Uw' 9Uw2 8Rw 9Fw2 L' 9Rw 9Bw2 8Rw' 9Dw2 B 9Lw' 8Rw 8Dw' 8Bw 9Lw 8Rw2 8Dw 8Uw' F 8Uw' 8Bw' 7Uw 16Lw2 7Fw D' 16Rw' D2 16Fw2 16Rw 16Uw2 7Bw' D2 U2 16Lw' 15Uw2 7Fw 7Rw' 15Bw2 15Lw' 15Fw2 U' 15Lw2 7Bw' U' 7Rw 15Bw2 14Uw2 14Lw2 14Rw2 14Uw2 R 14Fw2 L' 14Dw2 7Uw 14Lw D2 7Dw 14Rw' 7Uw' 13Rw' 13Fw2 13Lw2 13Uw2 13Bw2 7Fw' 13Uw2 F 13Lw 7Uw' R' 13Fw2 12Uw2 12Fw2 7Uw' F' 7Fw2 12Lw 12Uw2 B' 12Rw' 7Fw 12Lw U 7Lw' 11Rw2 11Bw2 L 11Dw2 7Uw' 7Fw 11Uw2 7Rw2 U2 7Uw' 11Bw2 7Lw' B 10Dw2 7Uw' 10Rw2 B 10Lw' 7Rw2 D2 7Uw' 10Rw B' 9Dw2 9Lw 7Fw2 9Lw2 9Uw2 F 7Dw 9Bw2 7Uw F2 7Fw 9Lw2 8Fw2 8Rw 8Bw R 8Rw2 8Dw2 8Rw 8Dw2 8Bw' 8Fw2 8Lw2 7Fw' 7Uw 8Rw' 8Fw2 8Rw 8Dw2 7Fw' 8Uw2 F 8Lw' 7Uw2 F' 7Rw2 7Fw' 7Lw' 7Dw' 6Uw' 6Bw 16Lw2 16Bw2 6Uw 16Lw B' 16Dw2 B' U2 6Uw' 16Lw2 6Uw' 15Rw F2 15Lw 15Dw2 F' 6Dw' 15Lw2 6Fw' 15Rw2 6Uw 6Fw 15Uw2 6Uw2 14Lw' 6Bw' 14Dw2 14Fw2 U 14Lw2 6Dw 14Rw F2 14Fw2 6Dw 14Rw 6Uw 13Fw2 13Rw' 13Bw2 13Lw2 6Fw 13Rw' 6Bw' 13Rw2 6Fw' 13Lw' 6Fw2 6Uw 13Lw' 12Rw' 12Fw2 12Rw B' 12Dw2 12Bw2 L' 6Fw 12Rw' 6Fw' 6Uw' B2 12Rw' 11Bw2 11Rw2 B2 F2 11Rw' 11Fw2 6Rw 11Dw2 R2 6Fw' D2 11Lw U2 10Rw 6Fw' 10Uw2 F' 6Uw' 10Fw2 10Dw2 6Rw 6Fw' D 10Lw' 9Uw2 6Fw' 9Rw2 D2 9Bw2 9Rw' D' 9Lw2 6Uw 9Lw2 9Dw2 B' 6Fw2 9Lw' 8Rw 8Uw2 8Fw2 8Lw2 6Fw 8Rw 6Uw2 8Rw2 8Uw2 8Rw 6Uw' 8Rw' 7Uw2 7Bw' L2 7Rw2 7Uw2 R' F 7Rw 7Dw2 7Fw' 7Uw2 6Fw2 7Rw 6Fw 7Rw2 F2 7Uw2 7Rw' 7Dw2 6Uw L' 6Fw' 7Lw2 6Uw2 6Fw' R2 6Fw' 6Dw' B' 6Uw' 16Dw2 16Fw2 R 5Dw 16Rw' U2 16Lw' 16Dw2 5Fw2 U' 16Rw' 16Fw2 5Bw 15Dw2 15Rw 5Uw' 15Fw2 15Lw 5Dw' 15Lw 5Bw 15Lw' U2 5Fw2 15Rw' 5Uw' 14Bw2 5Lw2 5Dw' 14Rw 14Bw2 14Rw' D' 14Dw2 5Lw' 5Uw' 14Lw 5Fw' 13Dw2 13Rw 5Dw' 13Lw2 13Dw2 5Fw2 13Lw2 D' 5Uw' 13Lw 12Uw2 12Rw2 5Uw 12Lw' 5Fw 5Rw 12Fw2 R2 5Rw2 5Dw' 12Rw' 5Fw' 11Lw2 11Bw2 11Fw2 11Uw2 11Rw 5Fw2 D 11Lw D2 5Uw 5Bw 11Lw' 5Uw' 5Bw D' 10Lw 10Rw2 10Bw2 5Rw' 10Dw2 5Fw 10Lw 5Uw2 F2 10Uw2 5Uw' 9Uw2 9Bw2 9Lw' 9Uw2 9Bw2 F2 5Dw' 9Lw' 9Bw2 5Uw2 5Fw 9Lw 9Fw2 5Uw' 5Bw2 8Lw2 8Dw2 B 8Lw' F' 8Dw2 5Uw' L' 5Dw 8Lw' 7Uw2 7Fw2 5Uw2 F 7Lw' 7Rw 7Dw2 R 7Bw2 7Dw2 5Bw' 5Lw D 7Lw 6Uw' 6Rw' 6Uw L' 6Uw' 6Lw2 R' 6Bw2 6Fw2 6Dw' 6Uw2 6Rw' 6Fw2 6Lw' 6Dw2 5Lw 6Rw' B' 6Lw D' 5Bw 6Lw' 5Uw 5Rw2 D 5Bw' 5Lw' B' 5Uw' U' 16Rw2 4Uw' 16Rw' D' 16Lw R' 16Bw2 D2 4Bw R' 16Bw2 4Uw' L' F 15Uw2 15Rw2 15Bw2 15Lw' 15Fw2 15Uw2 4Fw' D' B 15Lw' 14Uw2 14Rw' 14Bw2 4Uw 14Lw 14Dw2 F2 4Uw' 14Lw2 4Bw' D2 4Uw' 14Rw 13Bw2 F 13Lw' 13Bw2 13Fw2 4Uw2 4Fw' 13Uw2 B' 13Lw B2 4Uw' 13Lw' 4Uw R' 12Rw 12Dw2 12Uw2 4Lw' 12Rw' 4Bw 12Lw' 12Rw D2 4Rw' 12Bw2 11Uw2 11Rw2 4Fw' 11Lw 11Bw2 4Fw2 11Uw2 11Rw' B' 11Lw2 U2 4Uw' 11Rw' 10Uw2 4Rw2 10Fw2 10Dw2 U 10Rw 4Bw 10Lw2 F2 10Rw' 9Dw2 U' 9Lw 9Bw2 9Fw2 4Rw 9Fw2 L' U2 4Fw' 9Lw 8Uw2 F2 8Rw' 8Uw2 8Fw2 8Rw 4Uw' B 8Uw2 4Bw2 U 4Bw 8Lw' 4Uw2 7Fw2 4Uw 7Rw 7Bw2 U' 7Rw' 7Fw2 7Rw F 7Lw2 7Rw' 6Fw2 6Dw2 4Fw2 4Uw 6Rw 6Bw2 U2 4Uw' 6Lw 6Fw2 L' 4Rw 6Fw2 5Uw' 5Rw 5Uw L2 5Lw' R 5Uw' R 5Rw 5Uw' 4Bw' 4Rw 5Fw2 5Uw2 4Fw2 5Lw' 4Fw' 5Rw B 5Lw R' 5Dw2 4Uw2 4Bw' L' 4Lw 4Uw' 4Fw' 16Bw2 R 3Uw' 16Rw2 16Uw2 3Bw 16Uw2 3Fw' 3Dw B2 16Rw' U' 15Rw2 U 15Bw2 F2 15Dw2 15Bw2 3Uw' 15Lw B' 3Fw 15Lw' 14Uw2 L' U2 14Lw2 14Fw2 14Lw' 14Dw2 3Bw' 14Lw2 U' 3Uw' 14Rw 3Uw' 13Rw B2 13Rw' 13Dw2 13Lw' 3Rw2 13Uw2 3Rw2 U2 3Uw' 13Lw2 12Uw2 F2 12Lw2 3Rw 12Fw2 R2 12Rw2 12Dw2 3Bw 12Lw 3Uw' B 12Uw2 3Uw2 11Bw2 U 11Fw2 11Rw' 11Dw2 3Uw 11Bw2 11Lw2 D' B 11Rw' 10Uw2 F' 10Rw2 3Fw' 10Lw 10Bw2 F' 10Dw2 3Fw L B 10Uw2 9Rw2 9Uw2 F' 9Dw2 F2 9Rw2 F 3Uw 9Bw2 3Uw2 9Fw2 8Uw2 8Rw' 8Dw2 8Uw2 8Rw' 3Dw 3Fw' 8Lw R' 8Dw2 R 3Dw 8Rw' 3Uw2 7Fw2 7Uw2 B' 7Rw2 3Uw' 7Fw2 7Lw' 7Dw2 7Uw2 3Bw 7Bw2 F2 7Lw' 6Lw2 3Dw' 6Lw2 6Fw2 6Lw' 6Dw2 3Bw2 3Uw 6Lw2 6Rw' B2 6Rw 5Uw2 5Rw 5Bw2 3Fw' R' 5Fw2 U' 5Uw2 5Lw' F' 5Lw' 5Rw D2 4Bw 4Lw2 4Rw' 4Bw2 4Dw2 4Rw' U2 4Fw' 4Uw2 F 4Dw2 4Lw' 3Bw 4Lw 4Rw 3Uw 4Lw' 4Uw2 3Fw 4Rw 3Rw B' U' 3Bw' 3Dw 3Fw' 16Fw2 D' 16Bw2 2Fw 16Lw' R2 2Uw 16Fw2 2Rw2 2Fw' 16Lw2 D2 16Lw2 15Uw2 B2 15Rw' 15Dw2 U 15Rw' 15Uw2 15Lw2 2Bw 15Rw2 2Uw' 2Rw2 U2 15Lw' 2Fw' 14Rw' 2Uw' 14Fw2 U2 14Lw F' 14Uw2 2Bw' 2Lw2 U' 14Lw 13Bw2 13Lw 2Uw 13Bw2 13Rw 13Fw2 2Dw 13Bw2 L' 2Bw 2Dw' 13Rw' 2Dw F 12Uw2 2Rw' 12Rw 12Bw2 12Uw2 2Fw 12Lw2 U' 12Lw 11Fw2 2Dw' 11Rw2 F2 11Lw' 11Dw2 2Rw' 11Bw2 2Uw 2Rw2 2Bw' 11Lw' F 10Lw2 U' 10Bw2 10Dw2 2Bw 10Lw' 2Fw' 10Uw2 B' U2 10Rw' 9Fw2 9Rw U2 9Lw 2Fw 9Lw' U 9Bw2 L2 2Bw' 9Rw U 8Lw 8Bw2 F' 8Lw2 8Dw2 8Lw D2 8Fw2 2Dw 2Bw' D' 8Rw' F 7Lw' 7Fw2 7Rw' 7Uw2 B2 7Dw2 2Bw 2Uw' 7Lw' D 2Fw' 7Rw2 2Uw' 6Rw D 6Fw2 6Uw2 2Fw' 6Dw2 U 2Fw 6Dw2 5Uw2 5Rw' D 5Lw2 5Rw' 2Bw 5Lw 2Dw' 2Lw2 2Bw 5Lw 4Fw2 2Uw 4Uw2 4Lw' 4Fw2 2Rw' 4Uw2 2Rw 2Dw F2 4Lw' 3Fw' 3Uw2 3Fw2 3Lw' 3Bw2 D2 F 3Lw 3Fw' 2Uw' B' 2Dw 3Fw2 3Rw' 3Uw2 3Lw2 2Uw' 2Fw2 R' 2Uw 3Bw2 2Uw2 2Rw2 2Dw2 F' 2Uw B' F' 2Rw' 2Dw F 2Uw' U2 16Rw2 F' U 16Lw' U2 16Lw U' 16Rw' 16Fw2 16Lw2 16Rw2 16Dw2 15Lw F' 16Lw2 15Rw U 16Bw2 D' 16Fw2 15Rw 15Bw2 D' B 15Lw' D' 15Rw' 16Fw2 16Uw2 16Lw2 D B' 16Uw2 16Bw2 14Rw' 16Fw2 D' 14Lw B 16Rw2 F 15Fw2 15Lw2 15Bw2 15Rw2 D' 14Lw' F 15Lw2 14Fw2 14Lw2 U' 14Rw' B' F' 14Lw' 16Uw2 16Lw2 16Uw2 16Fw2 16Lw2 16Bw2 U' 13Rw2 F U' 16Rw2 D' 15Rw2 B' 15Lw2 F2 13Rw 15Bw2 13Rw 15Fw2 U' 15Bw2 F2 14Rw' 14Uw2 14Rw B2 14Fw2 14Uw2 14Rw 14Bw2 14Rw' 13Uw2 F' 14Lw2 14Bw2 U 14Uw2 14Rw2 14Dw2 U' 13Bw2 U' 14Rw2 B' F2 13Rw' D' F' 13Lw' 16Uw2 16Rw2 D' 12Rw 16Bw2 16Uw2 16Lw2 16Uw2 16Rw2 B F' 16Rw2 U' 15Bw2 12Lw' D 15Rw2 15Uw2 F 15Rw2 B 12Rw 15Bw2 U' 14Lw2 12Rw 14Bw2 14Uw2 12Rw2 U 14Rw2 B 14Dw2 13Uw2 13Rw' F' 13Rw2 U2 13Rw F 13Uw2 13Rw' 12Fw2 13Dw2 F 12Lw' 13Rw2 13Fw2 13Lw2 U 12Uw2 13Lw2 12Uw2 12Rw2 F' U' 12Rw' D' B 12Lw' 16Fw2 16Dw2 F 11Lw 16Fw2 D' 16Bw2 16Dw2 11Lw D' B 16Lw2 15Dw2 15Uw2 11Lw B' 15Lw2 15Rw2 15Bw2 D' B' 11Rw' 15Dw2 14Uw2 14Rw2 F' 14Dw2 U' 14Rw2 B' 14Dw2 14Fw2 11Rw 14Bw2 13Uw2 B' F2 13Lw2 D' 11Lw' 13Fw2 D' 13Bw2 13Fw2 F' 12Rw U2 12Rw' 12Bw2 12Rw 12Uw2 12Rw' 12Uw2 11Lw U' 12Fw2 12Rw2 F' 12Lw2 U' 12Rw2 11Fw2 11Lw' 11Bw2 F U 11Rw' 10Fw2 10Rw' 16Bw2 D 16Dw2 16Lw2 16Bw2 10Uw2 10Rw' F' U2 16Lw2 15Lw2 U 10Rw' 15Dw2 15Lw2 U2 10Lw' D' B 15Lw2 10Uw2 10Lw' B' 14Lw2 14Rw2 14Fw2 D' F' 10Rw' 14Dw2 13Rw2 D' 13Lw2 U 13Fw2 13Uw2 10Lw' 13Dw2 B 13Lw2 B 13Dw2 12Uw2 12Fw2 U' 12Rw2 F 12Lw2 10Rw' 12Bw2 10Rw2 F' 12Uw2 11Rw 11Uw2 11Rw' D U2 11Fw2 11Rw' 11Fw2 11Rw' B2 11Fw2 11Rw2 11Uw2 10Rw F' 11Rw2 U' 11Bw2 10Uw2 F' 10Lw D' 10Lw' 10Rw' 16Uw2 16Lw2 F' 9Rw' 16Rw2 16Bw2 U 9Lw B 16Dw2 U 16Bw2 15Bw2 15Uw2 F 9Lw' 15Lw2 15Bw2 9Lw 9Rw2 D 15Bw2 9Rw 15Fw2 F 14Uw2 B2 U' 14Rw2 D' B 9Rw 14Uw2 B' 13Uw2 F 13Lw2 B' 9Lw' 9Uw2 13Fw2 U 13Uw2 13Fw2 12Uw2 12Fw2 12Lw2 9Uw2 12Bw2 12Rw2 B' 12Dw2 12Uw2 9Lw U 12Rw2 U' 11Fw2 11Lw2 11Rw2 F' 9Uw2 9Lw 11Lw2 11Fw2 9Rw D B' 11Lw2 10Fw2 10Rw U2 10Uw2 10Rw' 10Uw2 F' 9Lw 10Lw2 10Dw2 9Rw2 B' 10Fw2 9Lw D' 10Fw2 9Uw2 9Rw D' B' 9Lw' U 9Rw' F' 8Rw' 16Rw2 16Fw2 8Rw2 16Dw2 U 16Bw2 F' 16Uw2 D 8Lw' 15Dw2 15Lw2 U 15Rw2 15Dw2 8Fw2 D' 8Dw2 B' 15Lw2 14Uw2 F U 14Rw2 U' 14Lw2 8Rw' 14Fw2 14Dw2 8Rw 14Fw2 13Dw2 F' 13Lw2 13Fw2 8Rw D B2 13Lw2 8Rw' 13Bw2 8Uw2 12Rw2 U' 12Uw2 12Lw2 8Dw2 12Rw2 12Fw2 8Rw' B 12Uw2 11Uw2 11Bw2 U' 11Lw2 B' 11Uw2 11Bw2 F' 11Uw2 8Rw 11Dw2 10Rw2 10Uw2 B 8Rw D 10Bw2 10Uw2 8Rw' B 10Rw2 9Uw2 9Rw' B2 F 9Rw 9Bw2 9Uw2 F' U2 9Rw' U' 9Fw2 U' 9Lw2 9Fw2 D 8Lw' F 9Rw2 U' 9Fw2 8Uw2 8Rw' U' 8Lw' D B' F' 8Rw' U 16Rw2 B' 7Rw' 16Bw2 16Dw2 B' 7Rw2 16Dw2 16Fw2 D' 16Fw2 D2 U' 15Lw2 D2 7Rw' 15Dw2 15Bw2 D' F' 15Rw2 14Bw2 14Uw2 F 14Rw2 B' 14Uw2 7Lw2 D' F 14Lw2 13Dw2 13Rw2 D' F' 13Uw2 7Lw' 13Fw2 7Rw B U 13Rw2 12Uw2 12Fw2 U2 7Lw 12Uw2 12Lw2 U 7Lw' 12Rw2 B 12Uw2 7Rw' D' 11Uw2 11Lw2 F' 11Dw2 11Uw2 7Lw2 F 11Uw2 10Rw2 7Fw2 U' 10Lw2 U 10Lw2 7Rw' 10Dw2 F 10Uw2 7Bw2 U' 9Fw2 9Uw2 9Lw2 9Bw2 D2 F U 9Rw2 8Rw B 8Fw2 8Rw' B' 8Fw2 8Rw' 8Uw2 B' U 7Uw2 8Lw2 8Uw2 7Fw2 U 8Bw2 F 8Lw2 F' 7Lw2 B' U' 7Lw' D' 7Rw' 16Bw2 16Lw2 6Rw 16Bw2 16Uw2 6Lw' D' F 16Rw2 15Lw2 6Rw' 15Uw2 15Lw2 D 15Bw2 F2 6Rw2 B' 15Rw2 14Fw2 U 6Rw 14Rw2 F' 14Fw2 14Rw2 B' 14Lw2 D' U' 6Rw' 14Uw2 13Uw2 6Rw 13Uw2 B' U 13Rw2 13Uw2 13Rw2 6Bw2 D 13Fw2 12Fw2 6Rw' 12Bw2 6Uw2 12Lw2 U' 6Rw' 12Uw2 6Rw' 12Bw2 11Fw2 D F 11Rw2 11Dw2 6Lw F U' 11Rw2 10Rw2 10Fw2 6Rw 10Dw2 6Rw 10Uw2 B 10Rw2 D 6Rw' U 10Fw2 B 6Fw2 9Rw2 9Dw2 U' 6Rw 9Dw2 6Lw B 9Dw2 6Rw 8Uw2 8Lw2 F' 6Rw U' 8Rw2 6Dw2 8Dw2 6Rw' 8Uw2 F 8Dw2 7Lw' D' F' 7Fw2 7Rw2 D F' 7Lw' F' 7Lw2 U 7Rw2 B' 7Lw2 B' 6Lw' 7Bw2 D' 7Lw2 6Uw2 6Lw D' F' U 6Lw' F2 16Rw2 16Fw2 16Rw2 U 16Bw2 16Uw2 5Lw' F 16Lw2 5Bw2 U' 15Lw2 5Rw B' 15Rw2 B' 15Fw2 15Uw2 5Lw U' 15Bw2 U' 14Rw2 U 14Lw2 D' 14Uw2 14Fw2 5Rw' F' 14Uw2 14Fw2 B2 13Uw2 13Bw2 13Lw2 B' 5Lw D' 13Fw2 13Dw2 5Rw 12Bw2 12Lw2 12Dw2 12Rw2 F2 5Lw' 12Bw2 U 12Rw2 B' 12Dw2 U 11Lw2 11Fw2 11Dw2 5Lw F' 11Uw2 11Fw2 D' 5Rw F' 11Lw2 10Dw2 10Fw2 5Lw 10Bw2 10Rw2 U 10Fw2 10Dw2 B 10Lw2 F' 10Dw2 9Uw2 B' 9Lw2 9Fw2 5Rw' 9Bw2 D 9Lw2 9Uw2 B' U2 5Uw2 9Lw2 U 5Lw2 8Lw2 8Fw2 8Rw2 D 5Rw 8Uw2 B 8Dw2 7Uw2 7Fw2 7Lw2 7Rw2 U' 5Lw' F 7Dw2 5Rw' 7Uw2 D' 6Rw 6Bw2 F' 6Rw2 F' 6Dw2 6Uw2 6Lw' 6Bw2 D' 6Lw2 6Bw2 6Fw2 5Dw2 F 6Uw2 B 6Dw2 5Lw' F' 5Rw2 D' B' 5Lw' 5Rw' U 16Lw2 16Uw2 B' U 4Rw 16Uw2 16Rw2 16Uw2 B 4Rw' 16Dw2 4Lw' U' 15Bw2 F 15Lw2 U 15Fw2 D' 15Fw2 4Rw2 F 15Rw2 F' 14Dw2 4Lw' F2 14Rw2 14Bw2 14Rw2 F' U' 14Fw2 14Dw2 4Fw2 U' 13Fw2 4Lw 13Bw2 13Rw2 U 13Rw2 13Uw2 4Rw' 12Dw2 F' 12Rw2 12Bw2 F2 12Fw2 4Lw' 12Bw2 U' 12Fw2 11Uw2 11Rw2 U 4Rw 11Bw2 11Uw2 11Lw2 4Fw2 4Lw 11Dw2 F 11Uw2 10Uw2 4Rw D' 4Rw' 10Dw2 B' F' 10Uw2 4Rw 10Dw2 U 9Bw2 9Fw2 9Uw2 F 9Rw2 4Uw2 9Rw2 F' 4Rw U' 9Rw2 8Uw2 4Lw2 8Rw2 D' 8Uw2 8Lw2 B' 4Rw 8Fw2 4Rw' 8Uw2 4Uw2 7Rw2 U' 7Bw2 7Lw2 7Fw2 7Rw2 7Uw2 7Rw2 B 7Rw2 U' 6Rw2 6Bw2 F' 6Rw2 6Dw2 F 6Rw2 4Fw2 U' 4Rw2 6Bw2 F2 5Lw 5Bw2 F 5Lw' B2 F' 5Lw' U2 5Lw2 4Fw2 4Rw' 5Bw2 4Lw 5Bw2 F' 5Uw2 4Fw2 4Rw F' 4Lw2 U' 4Lw' B' 4Rw' 3Rw 16Dw2 U' 16Rw2 U 16Lw2 U 16Bw2 F 16Fw2 16Uw2 15Dw2 15Uw2 3Rw2 U' 15Rw2 U2 15Fw2 U' 15Bw2 F 15Rw2 14Dw2 14Lw2 U' 14Rw2 F 3Rw 14Bw2 D 14Rw2 13Lw2 3Rw D' 13Uw2 13Fw2 13Rw2 13Bw2 3Rw2 F U' 13Lw2 12Dw2 U' 12Bw2 3Lw B 12Lw2 D' 12Fw2 3Rw 12Bw2 12Fw2 11Fw2 U2 11Rw2 D' 3Rw' 11Fw2 11Rw2 B' 11Dw2 3Rw2 U' 3Rw' 11Uw2 10Dw2 10Uw2 3Rw' 3Uw2 10Fw2 10Lw2 U 3Lw B' 10Dw2 U 9Lw2 9Bw2 9Uw2 F 9Lw2 U 9Fw2 3Lw' 9Bw2 F' U 9Lw2 8Uw2 8Rw2 D' B' 8Lw2 F U' 8Rw2 3Fw2 8Uw2 3Rw 8Bw2 7Uw2 F' 7Lw2 3Fw2 7Lw2 7Dw2 3Fw2 U' 7Rw2 U2 6Lw2 6Fw2 6Uw2 3Bw2 U 6Lw2 6Uw2 B 6Fw2 3Rw' 6Fw2 5Uw2 3Fw2 5Lw2 5Bw2 5Rw2 3Dw2 3Lw' D' 3Lw' 5Rw2 B 5Uw2 U 4Lw' B2 F' 4Lw' 4Bw2 4Rw2 F 4Lw' U' 4Fw2 4Dw2 B 3Rw 4Rw2 B 4Bw2 U' 4Lw2 3Rw2 4Dw2 3Uw2 3Bw2 3Lw' D B 3Rw' 16Uw2 2Rw2 16Fw2 16Dw2 16Lw2 D 16Rw2 2Uw2 B 16Fw2 2Rw 16Bw2 F 2Rw' U' 15Rw2 D 15Dw2 15Bw2 2Lw' B' 15Uw2 14Lw2 14Rw2 14Uw2 14Fw2 2Lw U 14Rw2 B 14Uw2 2Rw' 14Dw2 U 13Lw2 F' 13Fw2 13Dw2 B 13Lw2 13Bw2 2Rw' 13Dw2 U 13Bw2 U' 12Lw2 B' 12Uw2 2Rw 12Fw2 12Lw2 D 12Bw2 U' 2Rw 12Dw2 U2 2Lw' 11Bw2 D 11Rw2 F' 11Lw2 F' 11Dw2 U 11Fw2 11Uw2 10Bw2 10Dw2 F 10Lw2 U2 10Rw2 10Bw2 2Rw' D' 10Dw2 10Fw2 9Uw2 F 2Lw 9Uw2 B 9Lw2 D' 9Lw2 D' 2Rw' 9Bw2 8Uw2 8Lw2 2Rw 8Bw2 F' 8Dw2 8Uw2 8Rw2 D U2 2Rw B' 8Rw2 7Bw2 7Uw2 F' 7Rw2 U 2Rw' 7Dw2 7Uw2 2Lw2 D' 2Lw B' 7Lw2 6Lw2 6Dw2 6Lw2 2Rw' 6Uw2 6Bw2 D' F' 6Lw2 5Uw2 F' 2Lw2 2Rw 5Dw2 U 5Bw2 2Dw2 F 5Rw2 5Dw2 4Bw2 F2 4Rw2 4Dw2 4Lw2 F' 2Fw2 4Rw2 D 4Bw2 3Rw F' U2 3Lw2 3Rw D2 F 3Rw' 3Lw2 U 2Rw' 3Uw2 3Lw2 D F' 3Lw2 2Fw2 2Rw' 2Uw2 2Fw2 D' 2Lw2 D' B 2Fw2 2Lw' 16Dw2 R 15Dw2 R 16Bw2 R 16Uw2 16Bw2 15Dw2 R 16Dw2 15Rw2 D 15Bw2 16Lw2 16Fw2 D 15Rw2 15Bw2 16Bw2 D 16Rw2 16Bw2 B 15Rw2 16Uw2 F' 15Dw2 16Rw2 F 15Lw2 15Uw2 F2 16Dw2 B 16Rw2 16Uw2 14Bw2 16Uw2 16Fw2 14Uw2 14Bw2 16Fw2 L' 16Fw2 14Dw2 14Lw2 R2 16Bw2 16Lw2 14Bw2 D' 14Bw2 D 16Rw2 14Fw2 16Uw2 14Lw2 14Dw2 F 14Dw2 14Lw2 F 16Rw2 D2 14Dw2 16Lw2 16Dw2 B 14Rw2 14Uw2 R' 14Fw2 15Uw2 15Bw2 R' 14Uw2 15Fw2 14Uw2 R 15Uw2 15Fw2 14Lw2 D' 14Rw2 14Bw2 15Rw2 15Fw2 14Lw2 15Rw2 D' 15Fw2 15Uw2 B 15Dw2 B' 14Lw2 14Rw2 15Rw2 15Dw2 B' 15Lw2 15Uw2 15Lw2 15Uw2 14Lw2 16Uw2 R 13Fw2 13Dw2 R' 16Dw2 R 13Uw2 L 13Dw2 16Fw2 13Rw2 D' 13Lw2 D 13Fw2 16Fw2 D' 16Lw2 16Rw2 13Bw2 16Uw2 16Rw2 16Dw2 F' 13Uw2 F 13Dw2 16Uw2 16Lw2 F2 16Dw2 F U2 13Rw2 15Uw2 13Bw2 13Uw2 15Uw2 15Fw2 R 15Dw2 13Uw2 13Bw2 15Dw2 R' 13Fw2 13Rw2 15Bw2 15Fw2 D 13Lw2 D 15Bw2 13Fw2 15Lw2 13Bw2 13Rw2 F' 15Uw2 B U2 15Lw2 F 13Uw2 F 13Lw2 15Uw2 14Uw2 13Fw2 13Uw2 14Uw2 14Rw2 13Bw2 14Uw2 14Bw2 R' 14Fw2 14Uw2 14Fw2 13Rw2 13Bw2 D 14Bw2 14Lw2 D 13Fw2 D2 14Rw2 D 13Bw2 13Uw2 14Rw2 F2 14Dw2 13Rw2 B 14Uw2 R2 14Dw2 F 13Dw2 14Lw2 F' 14Rw2 16Uw2 R 12Fw2 L' 16Uw2 12Fw2 12Uw2 16Uw2 12Fw2 16Rw2 16Bw2 16Rw2 D' L2 12Bw2 D 16Lw2 16Bw2 16Rw2 F 12Uw2 B 12Rw2 F 16Lw2 12Dw2 12Lw2 16Lw2 D2 B 16Rw2 15Bw2 15Fw2 R' 12Bw2 12Fw2 R 15Fw2 R' 12Uw2 R' 15Fw2 15Rw2 D2 12Lw2 12Fw2 D' 12Bw2 15Fw2 15Rw2 D 15Fw2 15Rw2 B' 12Rw2 12Uw2 15Rw2 12Uw2 15Rw2 B' 12Uw2 15Rw2 U2 12Lw2 15Lw2 R' 14Bw2 12Uw2 R' 14Uw2 12Bw2 14Uw2 L 14Uw2 14Fw2 12Rw2 14Fw2 L2 14Bw2 14Rw2 12Bw2 14Lw2 D' 12Lw2 D' 12Uw2 B' 14Rw2 F2 12Lw2 14Uw2 F' 14Lw2 F2 U2 14Lw2 12Dw2 14Uw2 12Bw2 13Dw2 13Uw2 R 12Fw2 12Dw2 13Bw2 13Uw2 L' 13Fw2 D 12Rw2 D2 12Fw2 D' 13Lw2 D' 12Bw2 D2 13Fw2 B' 13Rw2 12Dw2 12Rw2 13Uw2 F' 12Uw2 13Lw2 12Rw2 F' 13Dw2 12Lw2 B 12Dw2 R 11Fw2 L 16Bw2 16Fw2 11Dw2 16Fw2 D' 11Fw2 16Lw2 16Rw2 11Fw2 16Lw2 16Rw2 D 11Bw2 16Bw2 16Lw2 11Rw2 11Dw2 F 11Dw2 16Rw2 11Dw2 11Lw2 16Rw2 F' U2 11Uw2 16Lw2 15Uw2 11Bw2 11Uw2 15Uw2 15Fw2 R 15Dw2 11Uw2 11Bw2 15Dw2 R' 15Lw2 D2 15Bw2 D' 15Fw2 11Lw2 11Fw2 15Fw2 D 15Rw2 11Bw2 F' 15Lw2 15Dw2 11Rw2 15Dw2 L2 11Dw2 B2 15Uw2 B' F2 11Dw2 15Rw2 14Fw2 14Uw2 11Bw2 14Uw2 R 11Bw2 R' 14Fw2 14Lw2 D' 14Rw2 11Bw2 14Rw2 11Bw2 11Lw2 D 11Rw2 11Bw2 14Uw2 B 11Rw2 F' 14Dw2 11Uw2 F 11Lw2 B' 14Rw2 14Uw2 14Rw2 14Dw2 11Rw2 13Uw2 13Bw2 11Uw2 R 13Fw2 11Dw2 11Bw2 13Fw2 L' 13Dw2 L2 13Rw2 11Fw2 D' 13Lw2 D2 13Rw2 13Bw2 D' F' 13Lw2 13Dw2 B' 13Lw2 11Dw2 13Rw2 B' 11Lw2 11Dw2 11Uw2 13Lw2 F 13Uw2 11Uw2 12Uw2 R 12Fw2 12Dw2 11Uw2 12Uw2 L 11Fw2 12Dw2 11Lw2 12Bw2 12Rw2 12Bw2 D 11Bw2 D 12Rw2 11Bw2 D2 12Fw2 12Rw2 F2 11Lw2 11Rw2 12Dw2 12Rw2 B2 11Uw2 B' 11Uw2 11Lw2 D2 12Uw2 12Lw2 10Uw2 16Uw2 R' 16Uw2 10Fw2 16Rw2 16Bw2 10Dw2 L' 10Uw2 10Rw2 D' 10Bw2 10Lw2 D 10Bw2 16Fw2 B 16Dw2 R2 10Rw2 16Dw2 B' 16Lw2 D2 10Lw2 15Bw2 10Uw2 15Bw2 15Dw2 R 10Fw2 15Rw2 10Bw2 L' 10Fw2 15Rw2 15Fw2 10Rw2 D' 15Lw2 D 10Lw2 10Bw2 10Rw2 D' U2 15Rw2 15Dw2 F2 10Dw2 10Rw2 B 10Dw2 F' 15Rw2 15Dw2 R2 10Dw2 14Fw2 R 10Dw2 14Fw2 14Rw2 14Dw2 10Fw2 R' 14Fw2 14Lw2 10Bw2 14Rw2 10Fw2 D' 10Bw2 14Bw2 10Rw2 14Bw2 D' 14Lw2 B' 14Lw2 B 10Lw2 14Rw2 F2 U2 10Lw2 14Dw2 B 14Dw2 F 10Rw2 10Fw2 R 13Fw2 13Uw2 R' 10Uw2 R 13Uw2 13Lw2 D2 10Rw2 D 10Bw2 D' 13Bw2 13Rw2 10Bw2 13Uw2 10Lw2 F 10Lw2 10Rw2 10Dw2 B' 10Dw2 13Rw2 U2 10Lw2 13Uw2 10Rw2 13Dw2 12Fw2 R' 10Fw2 10Dw2 R' 12Fw2 10Uw2 12Fw2 10Uw2 D 10Bw2 10Rw2 D' 12Bw2 D2 12Lw2 D' 12Bw2 D' 12Fw2 10Uw2 12Rw2 F2 12Dw2 10Rw2 B2 10Dw2 B' 10Rw2 B' 12Dw2 10Uw2 10Fw2 11Rw2 11Bw2 R 10Bw2 10Dw2 11Bw2 11Dw2 10Fw2 R' 10Fw2 D' 11Bw2 D 11Bw2 10Lw2 10Fw2 11Lw2 D' 11Bw2 10Fw2 D' 11Dw2 L2 11Rw2 B F2 11Lw2 F' 11Uw2 10Lw2 F 11Uw2 10Lw2 16Dw2 R' 9Uw2 R 9Bw2 9Dw2 16Dw2 16Fw2 9Dw2 R2 16Bw2 D 16Rw2 9Fw2 9Rw2 16Rw2 D 16Fw2 16Uw2 16Lw2 B 9Rw2 16Rw2 F' 9Dw2 B' R2 16Dw2 9Lw2 16Lw2 F' 16Rw2 15Fw2 R' 9Uw2 15Fw2 15Rw2 15Uw2 9Fw2 R' D 15Rw2 D2 9Rw2 15Bw2 9Lw2 15Bw2 D 9Fw2 15Fw2 F2 15Uw2 9Rw2 F 9Rw2 F' 15Rw2 B' F' 15Uw2 9Rw2 15Uw2 14Rw2 9Uw2 9Bw2 L 14Fw2 14Uw2 9Bw2 9Uw2 14Uw2 9Fw2 9Lw2 D' 9Lw2 D' 9Lw2 9Rw2 14Bw2 14Rw2 9Bw2 14Uw2 9Lw2 14Rw2 9Uw2 B' 9Lw2 F 14Dw2 14Lw2 D2 14Dw2 14Rw2 9Dw2 9Uw2 13Fw2 9Uw2 13Bw2 R' 9Uw2 9Bw2 R' 13Fw2 D2 9Bw2 D 9Rw2 D 13Bw2 9Rw2 13Bw2 13Rw2 9Bw2 13Lw2 13Rw2 9Uw2 B' 9Lw2 9Dw2 B2 13Rw2 9Dw2 9Rw2 13Rw2 B 13Rw2 R' 12Dw2 9Fw2 12Dw2 12Rw2 9Dw2 12Fw2 9Dw2 12Fw2 9Rw2 12Fw2 D 12Bw2 L2 12Lw2 12Bw2 9Fw2 D 12Fw2 F 9Dw2 B' 12Dw2 9Rw2 F 9Uw2 B' 12Lw2 12Uw2 9Rw2 D2 9Dw2 9Lw2 9Uw2 9Fw2 R 11Dw2 9Bw2 R' 9Uw2 9Fw2 9Dw2 L' 11Fw2 D 11Lw2 9Fw2 11Lw2 9Rw2 D' 9Bw2 L2 9Lw2 11Fw2 11Uw2 9Lw2 11Dw2 11Lw2 11Uw2 F' 9Dw2 B2 9Dw2 U2 9Rw2 F 11Lw2 9Rw2 9Uw2 10Rw2 10Uw2 R 10Uw2 R 10Fw2 R 10Uw2 9Fw2 D' 10Fw2 10Rw2 D' 9Fw2 10Fw2 D2 9Lw2 10Rw2 10Bw2 D' F2 10Uw2 R2 9Dw2 B' 9Rw2 U2 10Lw2 F' 10Lw2 10Uw2 9Rw2 10Rw2 R' 16Dw2 8Fw2 R' 8Uw2 8Fw2 L 8Uw2 16Rw2 16Fw2 8Lw2 16Fw2 16Lw2 8Fw2 8Lw2 D' 8Bw2 D' 16Uw2 16Rw2 B 8Uw2 F' 16Uw2 16Lw2 16Uw2 8Rw2 16Dw2 B' F 16Uw2 15Uw2 15Fw2 R 8Uw2 R 8Fw2 15Dw2 R 8Dw2 L 15Dw2 D' L2 8Bw2 D' 15Rw2 8Fw2 8Lw2 D 8Lw2 D 15Bw2 15Lw2 B' 8Dw2 F2 8Lw2 U2 8Rw2 8Uw2 F 8Lw2 15Rw2 14Rw2 14Uw2 R 8Uw2 R' 14Dw2 R' 8Bw2 8Fw2 L' 14Fw2 8Rw2 8Bw2 D' 14Rw2 14Bw2 14Lw2 14Fw2 D 14Fw2 8Lw2 F' 8Lw2 F 14Uw2 F 14Dw2 8Lw2 14Dw2 B' 14Lw2 8Uw2 8Lw2 14Uw2 13Uw2 R' 13Fw2 8Uw2 8Fw2 13Fw2 L' 8Lw2 13Lw2 8Fw2 D 13Lw2 13Rw2 13Bw2 8Lw2 D 13Bw2 F2 13Dw2 8Lw2 13Uw2 F' 8Uw2 B 8Dw2 8Lw2 13Dw2 F2 13Lw2 12Uw2 R' 8Uw2 8Fw2 L' 12Fw2 8Uw2 12Bw2 8Uw2 12Fw2 12Lw2 D' 8Rw2 D' 8Rw2 12Rw2 8Fw2 12Fw2 8Rw2 12Fw2 12Uw2 F' 12Rw2 8Uw2 B' 8Dw2 F' 12Rw2 D2 B 12Rw2 8Uw2 12Lw2 12Dw2 8Uw2 11Rw2 11Bw2 R 8Bw2 R 11Bw2 8Uw2 8Fw2 R2 11Rw2 8Fw2 8Lw2 D' 11Fw2 D2 11Lw2 D' 8Uw2 F 8Lw2 F' 8Rw2 11Rw2 B' 8Lw2 11Lw2 D2 8Dw2 11Lw2 B 8Uw2 10Fw2 R 8Uw2 8Bw2 10Dw2 10Rw2 10Dw2 10Fw2 8Uw2 R' D' 8Lw2 8Bw2 10Bw2 10Fw2 8Lw2 D' 8Fw2 D2 10Fw2 8Uw2 B 8Dw2 8Uw2 B' 10Uw2 F' 8Dw2 B 10Uw2 10Lw2 8Dw2 9Dw2 R 9Fw2 9Uw2 8Fw2 R 8Uw2 8Bw2 9Uw2 L' 8Fw2 D 9Rw2 8Bw2 8Lw2 8Fw2 D 8Fw2 9Rw2 9Bw2 F2 8Lw2 B 9Uw2 B' 9Rw2 9Uw2 8Lw2 F 8Rw2 8Dw2 8Lw2 9Uw2 B2 9Lw2 7Uw2 7Bw2 16Dw2 R 16Dw2 7Fw2 16Dw2 7Bw2 L' 7Fw2 D 16Rw2 7Fw2 D2 16Bw2 7Rw2 D 16Lw2 D' R2 16Bw2 7Uw2 B' 16Lw2 7Uw2 7Lw2 F' 16Lw2 F 7Rw2 7Dw2 16Lw2 F' U2 16Rw2 15Uw2 7Fw2 15Dw2 7Uw2 7Fw2 R' 15Bw2 15Dw2 15Bw2 L' 7Bw2 15Lw2 D 15Lw2 15Fw2 D R2 7Rw2 15Bw2 15Lw2 7Bw2 U2 7Lw2 7Uw2 15Rw2 7Uw2 15Rw2 15Dw2 B' F' 15Dw2 7Uw2 R' 14Bw2 14Fw2 R' 7Uw2 7Bw2 L 14Dw2 7Fw2 14Uw2 14Rw2 7Fw2 14Lw2 14Fw2 7Rw2 D' 7Lw2 D 14Bw2 D 7Fw2 D' 14Rw2 14Uw2 7Lw2 B 7Uw2 7Lw2 F 7Rw2 U2 14Uw2 14Lw2 14Dw2 B' 7Lw2 7Uw2 13Rw2 13Fw2 7Dw2 R 7Uw2 7Fw2 R' 13Rw2 D 7Rw2 7Fw2 D' L2 R2 7Rw2 13Rw2 13Bw2 F' U2 7Rw2 13Uw2 F 7Rw2 F 13Lw2 B 13Dw2 F' 7Lw2 7Rw2 12Fw2 R' 7Fw2 7Dw2 12Fw2 7Dw2 12Fw2 R' 12Bw2 12Lw2 D L2 7Bw2 7Fw2 D' 12Lw2 12Bw2 7Uw2 F 7Dw2 7Uw2 7Lw2 F 12Lw2 B' 7Dw2 B U2 12Rw2 11Fw2 R' 7Uw2 11Fw2 7Uw2 11Fw2 7Dw2 7Fw2 R' 11Fw2 D' 7Bw2 D' 11Fw2 11Rw2 D 11Lw2 D 7Bw2 7Lw2 11Fw2 7Uw2 11Uw2 7Rw2 F2 11Rw2 U2 11Lw2 B' 7Lw2 7Uw2 B F2 11Uw2 10Bw2 10Fw2 R 7Bw2 7Dw2 R' 10Fw2 7Lw2 D 7Fw2 D' L2 10Bw2 D' 7Dw2 F 7Lw2 F2 10Rw2 10Uw2 R2 7Rw2 U2 7Uw2 10Uw2 10Rw2 7Bw2 9Uw2 7Bw2 R' 7Uw2 L' 7Bw2 9Bw2 7Rw2 D 7Fw2 D2 7Fw2 D 9Bw2 9Uw2 9Lw2 7Uw2 7Rw2 U2 9Lw2 7Uw2 F2 9Dw2 U2 7Lw2 9Dw2 8Dw2 R' 8Bw2 8Rw2 8Uw2 R 7Bw2 7Dw2 8Uw2 L' 7Bw2 8Bw2 D' 7Bw2 7Lw2 8Rw2 8Bw2 D' 8Rw2 8Bw2 D' 7Uw2 8Rw2 8Uw2 8Rw2 7Uw2 F 8Dw2 B F 8Dw2 7Rw2 B2 7Uw2 8Uw2 R 6Uw2 16Fw2 R 6Uw2 6Fw2 R' D' 6Rw2 6Fw2 D2 6Bw2 D 6Lw2 16Bw2 F2 16Rw2 F' 16Dw2 16Lw2 6Uw2 6Lw2 16Lw2 16Dw2 6Rw2 B 16Uw2 F 6Lw2 R' 6Uw2 15Fw2 R' 6Uw2 R 15Bw2 6Uw2 D 6Rw2 15Bw2 6Lw2 6Bw2 15Bw2 D' 6Bw2 6Lw2 15Fw2 F 15Rw2 6Dw2 15Dw2 6Lw2 F' 6Lw2 B 15Rw2 D2 6Lw2 6Uw2 15Uw2 14Uw2 6Fw2 R 6Bw2 6Fw2 R' 6Bw2 14Uw2 14Rw2 14Fw2 D2 6Fw2 14Lw2 D2 6Lw2 D 6Bw2 D' 14Fw2 14Uw2 F' 6Dw2 F 6Dw2 6Rw2 14Uw2 14Rw2 B' F 14Uw2 6Lw2 13Uw2 6Fw2 6Uw2 13Uw2 13Bw2 R' 13Dw2 6Uw2 6Fw2 13Dw2 R' D' 6Fw2 6Lw2 13Bw2 6Rw2 13Bw2 D' 13Fw2 13Lw2 6Bw2 13Uw2 13Rw2 F' 6Lw2 B 6Lw2 13Lw2 R2 F2 6Uw2 13Lw2 13Uw2 6Rw2 12Bw2 6Uw2 12Bw2 12Dw2 R 6Fw2 L' 12Fw2 6Rw2 6Fw2 D 6Fw2 12Lw2 6Fw2 12Lw2 D' 12Dw2 F 6Dw2 F' 12Rw2 B F 12Rw2 6Dw2 12Dw2 6Rw2 R' 11Dw2 6Fw2 11Dw2 11Rw2 6Dw2 11Fw2 6Dw2 D2 R2 6Rw2 11Fw2 6Rw2 D 6Fw2 6Rw2 D' 6Uw2 F 11Dw2 6Lw2 11Lw2 F 6Uw2 11Rw2 B2 6Dw2 6Uw2 6Lw2 11Uw2 R 10Uw2 10Bw2 6Uw2 10Rw2 6Bw2 R 6Uw2 L 10Uw2 6Fw2 6Rw2 6Bw2 6Fw2 D' 6Fw2 D' 6Rw2 10Fw2 F2 10Rw2 F2 10Rw2 6Uw2 B' 6Dw2 10Lw2 6Dw2 6Lw2 10Lw2 6Uw2 9Rw2 9Uw2 6Fw2 9Uw2 9Bw2 R 6Uw2 6Bw2 6Dw2 R' 6Fw2 9Rw2 D 6Bw2 6Fw2 6Lw2 D' 9Rw2 6Fw2 6Rw2 9Fw2 9Rw2 6Dw2 9Dw2 B' 6Rw2 F' 9Lw2 6Uw2 R2 9Dw2 9Lw2 9Dw2 6Lw2 R' 8Bw2 6Dw2 8Bw2 8Uw2 R' 6Fw2 L' 8Fw2 D 8Fw2 6Lw2 D 6Bw2 6Rw2 D2 6Rw2 D 8Fw2 8Uw2 8Lw2 F2 8Dw2 B 8Rw2 6Uw2 L2 8Rw2 6Dw2 6Lw2 6Rw2 F 8Dw2 6Uw2 7Fw2 6Uw2 7Rw2 6Fw2 7Uw2 6Bw2 R' 7Uw2 6Lw2 7Fw2 7Lw2 D 7Lw2 7Rw2 7Bw2 D2 6Rw2 D' 7Uw2 7Lw2 6Rw2 7Uw2 F 6Uw2 7Lw2 6Dw2 6Rw2 7Dw2 7Uw2 F 6Dw2 B2 7Lw2 16Fw2 5Dw2 R' 5Dw2 16Rw2 5Bw2 R' 5Fw2 5Rw2 D' 5Fw2 16Rw2 5Fw2 5Lw2 16Lw2 5Fw2 D 5Bw2 16Fw2 16Uw2 5Lw2 16Rw2 5Uw2 16Rw2 F' U2 5Lw2 F 16Uw2 B2 5Dw2 15Rw2 15Uw2 5Bw2 R 5Bw2 5Fw2 L 5Dw2 5Fw2 15Rw2 5Bw2 5Fw2 D R2 15Bw2 5Fw2 15Lw2 5Bw2 15Bw2 D' 15Uw2 F 5Uw2 B 15Rw2 B2 5Lw2 F2 5Dw2 B2 15Dw2 B2 15Lw2 14Fw2 14Rw2 14Uw2 5Bw2 R 14Bw2 R' 5Bw2 14Uw2 5Fw2 14Rw2 D2 14Rw2 D 5Lw2 14Lw2 D 5Fw2 5Rw2 14Fw2 14Dw2 14Uw2 5Lw2 F' 14Uw2 B2 14Dw2 U2 5Uw2 5Lw2 14Lw2 5Uw2 B' 14Lw2 13Uw2 R 5Uw2 R' 13Dw2 R 5Bw2 L' 13Rw2 13Fw2 13Lw2 13Rw2 13Fw2 D 5Bw2 D' 5Bw2 5Rw2 B' U2 5Rw2 F2 5Lw2 F' 13Dw2 13Rw2 F' 5Rw2 B 13Dw2 5Uw2 R' 5Fw2 R 12Uw2 5Fw2 12Uw2 12Bw2 12Fw2 5Uw2 5Fw2 12Rw2 D2 12Rw2 12Bw2 5Rw2 D' 5Bw2 12Rw2 D 12Fw2 5Uw2 B 12Rw2 5Uw2 F 12Rw2 F' 5Uw2 12Lw2 B U2 12Lw2 5Rw2 11Fw2 R' 5Uw2 11Rw2 5Bw2 5Dw2 R' 5Uw2 5Rw2 5Fw2 L2 5Bw2 D' 11Rw2 11Fw2 D' 5Lw2 5Fw2 11Fw2 11Dw2 5Lw2 5Dw2 F' 5Uw2 B 11Rw2 11Uw2 5Rw2 D2 B 5Lw2 10Dw2 5Uw2 5Fw2 R' 10Dw2 5Bw2 10Rw2 10Fw2 L' 5Fw2 10Fw2 D' 5Fw2 10Lw2 5Bw2 10Lw2 D 5Rw2 5Bw2 5Rw2 B' 10Uw2 R2 5Dw2 10Dw2 F' 5Uw2 5Rw2 10Rw2 10Uw2 5Lw2 F2 10Dw2 R' 9Uw2 9Bw2 5Uw2 R' 5Uw2 L 9Uw2 5Fw2 9Fw2 5Lw2 9Bw2 9Lw2 D' 9Fw2 D 5Bw2 D2 5Rw2 9Fw2 F2 5Uw2 9Rw2 5Uw2 B' 9Uw2 5Rw2 9Uw2 B' 5Rw2 5Uw2 F2 9Uw2 R' 8Uw2 5Fw2 8Uw2 8Bw2 R' 5Uw2 5Bw2 5Fw2 R' R2 5Rw2 8Bw2 D' 5Fw2 D' 5Lw2 5Bw2 8Fw2 8Rw2 5Bw2 5Fw2 5Rw2 8Dw2 5Rw2 8Uw2 8Lw2 5Uw2 F' 5Uw2 L2 5Lw2 F 8Uw2 5Fw2 R' 7Dw2 5Uw2 R 5Uw2 5Bw2 5Fw2 L' 5Fw2 5Rw2 7Bw2 D' 5Lw2 5Bw2 D 7Bw2 7Lw2 5Fw2 5Rw2 B 7Uw2 5Lw2 7Dw2 5Uw2 5Lw2 B 7Uw2 5Lw2 7Rw2 R 6Bw2 6Uw2 5Fw2 5Dw2 L 6Bw2 5Dw2 6Uw2 5Rw2 5Fw2 D' 6Lw2 5Rw2 D 6Lw2 6Bw2 6Rw2 6Fw2 D2 5Bw2 6Rw2 F 6Rw2 6Dw2 6Lw2 5Dw2 F2 5Lw2 6Dw2 B' 5Rw2 F' 6Rw2 4Fw2 R 16Uw2 4Bw2 16Dw2 R' 4Uw2 4Fw2 L' 16Lw2 4Bw2 D2 16Lw2 16Fw2 4Rw2 4Fw2 D' 4Lw2 D 16Fw2 4Lw2 F' 4Lw2 F' 16Dw2 B F 16Lw2 4Rw2 16Uw2 B' 4Rw2 15Fw2 4Dw2 15Fw2 15Uw2 R' 15Uw2 4Bw2 4Uw2 15Uw2 L' 15Fw2 15Lw2 4Fw2 D2 15Fw2 4Rw2 D 4Lw2 R2 4Bw2 D' 15Dw2 B' 4Uw2 F' 15Rw2 4Uw2 B' 4Dw2 B U2 15Rw2 4Dw2 15Dw2 14Fw2 14Rw2 14Uw2 4Bw2 R 14Uw2 R' 4Fw2 14Uw2 4Rw2 14Rw2 D2 14Rw2 D 14Lw2 14Fw2 D2 4Rw2 14Bw2 D' F' 4Dw2 14Uw2 R2 F' 14Uw2 4Lw2 F' 14Dw2 4Rw2 B' F2 4Uw2 14Lw2 13Uw2 R' 4Uw2 R 13Dw2 R' 4Bw2 4Dw2 L' 4Rw2 D2 13Fw2 13Rw2 4Bw2 4Lw2 D' 4Rw2 D 13Rw2 13Fw2 4Dw2 13Lw2 F 4Rw2 4Dw2 B 4Lw2 13Rw2 4Dw2 L2 13Lw2 4Uw2 12Fw2 R 4Bw2 4Dw2 12Rw2 4Uw2 R' 4Dw2 4Fw2 R2 12Rw2 4Fw2 D2 4Lw2 12Bw2 4Fw2 4Uw2 12Lw2 F 4Dw2 12Lw2 F' 4Uw2 F 12Lw2 B2 4Lw2 R 11Uw2 R' 4Fw2 4Dw2 11Fw2 4Uw2 11Rw2 11Uw2 L' D 4Lw2 11Bw2 11Rw2 4Bw2 D 4Bw2 4Fw2 D2 11Fw2 11Uw2 F2 11Dw2 B U2 4Rw2 F 11Lw2 11Uw2 4Lw2 F2 4Uw2 L2 11Dw2 R' 10Fw2 R 4Uw2 10Rw2 4Fw2 4Dw2 R' 4Fw2 4Lw2 10Bw2 D' 4Lw2 4Bw2 D 10Lw2 10Rw2 4Fw2 10Dw2 4Rw2 10Dw2 B 10Dw2 4Rw2 D2 10Dw2 4Lw2 B 10Rw2 F 4Lw2 9Fw2 4Uw2 R' 4Uw2 L' 9Fw2 4Uw2 9Rw2 4Bw2 9Uw2 D' R2 4Fw2 4Lw2 D 9Fw2 D 4Lw2 D' 9Fw2 F 4Rw2 F' 9Lw2 B' F' 4Uw2 4Lw2 9Rw2 F 4Dw2 R' 8Uw2 R' 4Uw2 4Fw2 8Rw2 4Uw2 L' 4Bw2 D 4Bw2 4Rw2 4Fw2 D' 8Bw2 4Fw2 4Rw2 8Bw2 4Rw2 F' 4Rw2 8Uw2 4Lw2 8Uw2 F 4Uw2 8Rw2 B2 8Dw2 4Bw2 7Rw2 7Uw2 R 4Dw2 4Uw2 L' 4Dw2 4Fw2 D' 4Bw2 D' 4Fw2 7Lw2 7Fw2 4Rw2 4Bw2 7Fw2 4Lw2 4Uw2 7Uw2 7Rw2 F U2 7Lw2 B 4Dw2 F' 7Dw2 4Uw2 4Bw2 R' 6Fw2 6Uw2 6Bw2 4Uw2 4Fw2 6Fw2 6Uw2 4Fw2 6Rw2 D 6Bw2 4Rw2 4Bw2 6Bw2 4Rw2 D' 4Rw2 F' 4Dw2 4Rw2 6Rw2 F2 6Uw2 F' 4Lw2 F2 6Lw2 4Dw2 6Uw2 R' 5Bw2 5Dw2 4Uw2 4Fw2 L' 5Dw2 5Bw2 4Uw2 4Rw2 5Bw2 D 5Fw2 D 5Rw2 4Bw2 D' 5Lw2 D 4Bw2 5Fw2 4Lw2 5Lw2 4Uw2 4Rw2 F 4Lw2 F 4Dw2 5Uw2 L2 5Dw2 4Lw2 5Uw2 5Rw2 3Fw2 16Uw2 3Fw2 16Dw2 R' 3Fw2 3Dw2 3Bw2 L' 16Fw2 D' 3Rw2 D' 3Lw2 16Rw2 D' 3Bw2 3Rw2 D' 16Fw2 U2 16Lw2 16Uw2 3Lw2 3Dw2 16Dw2 F2 16Dw2 F' 3Uw2 3Rw2 F' 16Dw2 15Rw2 3Fw2 15Dw2 15Uw2 R' 3Dw2 15Uw2 15Fw2 15Uw2 L' 3Fw2 D 15Fw2 3Rw2 D' 3Bw2 D 15Fw2 D 3Fw2 3Rw2 15Bw2 15Uw2 15Lw2 B 3Dw2 3Uw2 3Rw2 F U2 15Rw2 15Uw2 3Rw2 15Dw2 B' 3Rw2 R' 14Uw2 14Fw2 R 3Bw2 3Fw2 L' 3Fw2 14Uw2 3Fw2 R2 14Fw2 D' 3Rw2 D' 14Rw2 3Fw2 14Rw2 14Bw2 3Uw2 3Rw2 14Rw2 F 3Uw2 14Uw2 L2 3Rw2 14Dw2 14Rw2 U2 14Lw2 R' 3Uw2 13Bw2 13Fw2 13Uw2 3Fw2 3Uw2 13Uw2 13Bw2 D 3Rw2 D' 3Bw2 3Rw2 13Fw2 13Rw2 3Bw2 13Rw2 D' U2 13Rw2 B 3Uw2 B 13Rw2 B' 3Uw2 F 13Rw2 3Uw2 3Lw2 12Rw2 12Fw2 12Uw2 3Bw2 12Uw2 R 3Fw2 R' D' 12Lw2 3Rw2 3Fw2 12Rw2 D 12Bw2 12Fw2 12Rw2 12Bw2 D' 12Uw2 F 3Lw2 12Uw2 3Lw2 12Lw2 3Dw2 R2 12Uw2 F 12Rw2 12Dw2 12Uw2 11Rw2 11Fw2 3Uw2 3Bw2 R' 11Fw2 11Rw2 11Uw2 3Bw2 R' 3Bw2 D' 3Fw2 11Lw2 3Fw2 3Lw2 11Rw2 D R2 11Fw2 3Uw2 11Lw2 3Rw2 3Dw2 11Dw2 3Uw2 B' 3Lw2 B' 3Lw2 11Dw2 3Uw2 11Lw2 10Uw2 10Fw2 3Dw2 10Fw2 R' 3Uw2 10Rw2 3Bw2 L' D' 3Fw2 10Lw2 3Bw2 D 10Rw2 D' 10Lw2 3Rw2 D 10Bw2 F 3Rw2 3Uw2 3Lw2 F 10Uw2 B2 10Uw2 10Lw2 D2 B' 3Uw2 B 10Rw2 9Rw2 3Uw2 R 9Dw2 R 3Uw2 3Bw2 L' 9Bw2 3Dw2 3Lw2 D' 3Lw2 3Bw2 9Fw2 D' 9Rw2 3Bw2 9Rw2 9Bw2 B' 3Uw2 3Rw2 F' 9Rw2 3Uw2 9Rw2 9Uw2 3Rw2 F2 9Dw2 9Rw2 9Uw2 8Rw2 8Dw2 R 3Fw2 3Dw2 L' 8Fw2 D' 8Fw2 3Rw2 8Fw2 D2 3Rw2 D' 8Fw2 3Uw2 8Uw2 3Rw2 F' 8Rw2 3Uw2 F 8Lw2 B2 8Lw2 8Uw2 3Rw2 3Uw2 7Bw2 R' 7Uw2 3Bw2 7Rw2 7Bw2 R' 7Bw2 D 7Rw2 D 3Rw2 D' 7Fw2 D' 3Bw2 D2 7Fw2 7Lw2 7Uw2 F2 3Rw2 F' 3Dw2 F' 7Dw2 7Lw2 D2 7Dw2 F 3Rw2 3Bw2 6Uw2 6Rw2 6Bw2 R 6Bw2 R 3Bw2 6Uw2 6Fw2 L2 6Fw2 6Lw2 6Rw2 D' 3Bw2 3Lw2 D 3Bw2 F' 6Rw2 F 6Lw2 6Rw2 F 3Rw2 U2 3Uw2 3Lw2 6Lw2 3Uw2 B 6Rw2 3Uw2 5Fw2 5Uw2 3Bw2 3Uw2 5Uw2 5Fw2 3Lw2 3Bw2 5Bw2 D 3Bw2 D 5Bw2 3Fw2 5Rw2 3Fw2 3Lw2 5Dw2 U2 5Lw2 3Rw2 B' 3Rw2 F' 3Uw2 3Lw2 5Dw2 4Uw2 4Fw2 3Uw2 3Fw2 R 3Uw2 4Fw2 4Uw2 3Rw2 3Fw2 D2 3Lw2 4Bw2 4Rw2 4Bw2 D 4Rw2 D 4Lw2 3Fw2 4Uw2 4Rw2 B 4Lw2 F' 4Rw2 B2 4Uw2 F 3Rw2 4Dw2 3Lw2 16Uw2 16Rw2 16Dw2 16Uw2 R 2Uw2 2Fw2 L' 16Fw2 D2 2Rw2 2Fw2 16Rw2 16Fw2 D 2Fw2 D 2Lw2 16Fw2 F2 2Dw2 16Dw2 B 16Rw2 F2 16Uw2 B' U2 16Rw2 16Dw2 2Rw2 F 2Uw2 15Dw2 2Uw2 R' 2Uw2 2Fw2 15Rw2 2Uw2 L' 15Fw2 2Lw2 D 2Bw2 L2 15Fw2 D 2Bw2 D2 15Fw2 2Rw2 15Dw2 2Lw2 15Lw2 B 2Lw2 F2 2Rw2 F' 15Uw2 14Dw2 2Uw2 R' 14Bw2 14Rw2 14Dw2 2Bw2 2Dw2 L' 14Rw2 2Fw2 2Lw2 14Bw2 14Rw2 D 2Lw2 D' 14Lw2 2Fw2 2Uw2 14Uw2 2Rw2 B' 2Lw2 B' R2 14Rw2 14Dw2 F2 2Dw2 14Lw2 2Rw2 2Fw2 R 13Fw2 R 2Uw2 2Fw2 R' 13Bw2 2Uw2 13Bw2 D' 2Rw2 2Bw2 2Fw2 D 13Fw2 2Lw2 13Bw2 F2 2Uw2 2Lw2 13Uw2 2Rw2 F' U2 2Rw2 13Dw2 2Lw2 2Dw2 13Dw2 F 13Uw2 12Fw2 R' 2Dw2 12Fw2 2Dw2 12Fw2 R' 12Fw2 R2 12Fw2 12Lw2 12Rw2 D' 2Lw2 D' F' 2Rw2 F' 2Rw2 2Dw2 12Dw2 B 2Dw2 B' 12Dw2 U2 12Lw2 12Rw2 11Uw2 2Fw2 2Uw2 2Bw2 11Uw2 11Fw2 2Uw2 2Fw2 D 11Bw2 2Lw2 D 11Rw2 11Bw2 2Fw2 D' 11Fw2 D' 11Rw2 11Uw2 2Rw2 F 2Rw2 B' 11Lw2 F' U2 2Lw2 F' 11Uw2 10Uw2 2Fw2 10Uw2 10Fw2 2Dw2 R 2Dw2 R' 10Bw2 2Dw2 10Fw2 D2 10Lw2 D 10Rw2 D 2Rw2 D' 10Lw2 2Fw2 D 10Bw2 10Uw2 F 10Lw2 10Dw2 2Lw2 F' 10Lw2 2Rw2 10Rw2 2Dw2 2Lw2 B2 F' 10Uw2 R' 2Uw2 9Fw2 2Uw2 9Bw2 R 2Bw2 2Dw2 R' 9Fw2 D' 2Lw2 2Bw2 2Fw2 D' 9Bw2 2Rw2 9Fw2 2Rw2 9Dw2 F 2Rw2 2Uw2 F 9Dw2 2Lw2 B2 9Uw2 8Bw2 R' 2Uw2 2Bw2 8Rw2 2Bw2 2Dw2 R' 2Rw2 D' 8Rw2 D' 2Rw2 2Bw2 D2 2Lw2 8Fw2 2Dw2 8Lw2 F' 2Rw2 F 2Uw2 8Rw2 B2 2Lw2 8Lw2 R' 2Uw2 7Fw2 2Uw2 7Bw2 R 2Uw2 2Fw2 R' 2Lw2 D2 7Rw2 D 2Bw2 7Lw2 7Bw2 D 7Fw2 7Lw2 2Fw2 F' 2Lw2 2Uw2 B' 7Uw2 2Lw2 7Dw2 2Rw2 F2 7Dw2 7Lw2 2Uw2 7Lw2 6Rw2 6Uw2 2Fw2 R' 2Uw2 2Bw2 2Fw2 L' 2Dw2 6Bw2 D R2 2Bw2 6Lw2 D 2Bw2 D2 2Lw2 6Bw2 6Rw2 B 2Dw2 F' 6Uw2 B2 2Dw2 F2 2Lw2 F2 6Lw2 6Uw2 2Fw2 R' 5Dw2 5Uw2 2Fw2 R' 5Dw2 5Fw2 5Dw2 L' 5Fw2 D2 5Rw2 D 2Lw2 D' 2Bw2 5Rw2 2Fw2 R2 5Fw2 5Uw2 F2 2Rw2 5Dw2 B 2Lw2 F' 5Uw2 B2 5Dw2 5Uw2 F' 2Rw2 R' 2Uw2 4Uw2 4Bw2 4Rw2 2Fw2 R' 2Bw2 L' 2Fw2 4Dw2 4Fw2 4Lw2 2Rw2 4Fw2 D 2Fw2 D 4Bw2 2Lw2 4Lw2 2Bw2 4Dw2 2Rw2 4Uw2 F' 2Lw2 B' 4Uw2 2Lw2 B F' 4Lw2 2Rw2 4Rw2 3Uw2 2Bw2 R' 3Fw2 3Uw2 2Fw2 3Fw2 3Rw2 3Bw2 L' 3Rw2 3Fw2 D 3Bw2 2Lw2 D2 2Bw2 D' 2Lw2 3Rw2 D 3Bw2 B 2Rw2 3Dw2 2Rw2 D2 F 2Dw2 3Uw2 2Lw2 3Lw2 2Uw2 3Rw2 B 3Dw2 D' 16Rw2 16Uw2 R' U 16Lw2 D' L' F' B 16Uw2 L2 U2 F' 16Rw2 B R2 F 16Rw2 F' 16Dw2 U' 16Lw2 F2 16Lw2 16Fw2 16Lw2 U2 16Bw2 U' 16Fw2 U' R2 D 16Lw2 B2 16Rw2 R' U2 15Rw2 15Fw2 L' F' D2 L' B' U' U2 15Uw2 F' 15Uw2 L2 R2 15Rw2 B 15Rw2 F2 15Dw2 B 15Dw2 15Rw2 U2 15Lw2 R2 15Bw2 U 15Rw2 U' B2 15Fw2 U' L2 D R2 15Bw2 15Rw2 U2 14Lw2 D' 14Bw2 U B' U 14Bw2 R' 14Lw2 U' 14Dw2 14Lw2 R2 B L2 R2 B 14Dw2 14Fw2 R2 14Rw2 B 14Bw2 F2 R2 14Bw2 U' 14Rw2 U2 14Lw2 U R2 14Fw2 14Rw2 U2 14Bw2 14Lw2 U' F' R D 13Dw2 F' 13Dw2 L' B' D' 13Dw2 U2 B' L2 13Rw2 D2 B 13Lw2 U2 R2 D2 B 13Dw2 13Fw2 13Rw2 13Bw2 U' 13Lw2 R2 D 13Bw2 L2 B2 13Lw2 U 13Lw2 D' 13Fw2 U2 12Fw2 R B2 R' D 12Fw2 R' 12Rw2 U' L2 12Uw2 F' U2 L2 B' L2 12Uw2 F' 12Rw2 12Bw2 12Uw2 F 12Uw2 B2 L2 U' F2 U2 12Lw2 U' F2 R2 U' 12Lw2 R2 D' 12Rw2 12Fw2 B D 11Rw2 F' 11Uw2 B2 11Uw2 R' 11Rw2 U 11Lw2 B' R2 11Uw2 B' D2 11Lw2 F' 11Uw2 B' L2 11Uw2 U2 L2 11Lw2 D B2 11Bw2 D' R2 11Bw2 U' 11Lw2 U 11Lw2 11Rw2 11Bw2 11Lw2 10Uw2 10Lw2 R2 10Uw2 10Fw2 D' 10Bw2 R' D' U 10Dw2 10Rw2 10Dw2 10Rw2 10Uw2 F' R2 U2 10Lw2 10Fw2 10Dw2 B' 10Dw2 U2 F2 R2 10Rw2 U' 10Bw2 F2 10Lw2 10Rw2 U' R2 U' B2 10Rw2 9Dw2 9Rw2 9Fw2 D' 9Lw2 R' B' L' 9Rw2 U 9Lw2 F2 D2 9Lw2 9Uw2 F' L2 9Lw2 B2 9Uw2 B 9Rw2 9Dw2 U2 R2 U2 9Lw2 D' 9Lw2 F2 U R2 D 9Lw2 B2 U' F2 9Fw2 9Rw2 L 8Fw2 U' F' 8Fw2 D' R' F2 D' F' U2 B 8Uw2 R2 F2 8Uw2 B L2 B 8Dw2 L2 8Rw2 U 8Fw2 R2 B2 U 8Fw2 U' B2 8Rw2 U2 8Fw2 7Rw2 U 7Fw2 L' F 7Lw2 D' L' U' B U2 7Rw2 7Uw2 F 7Rw2 U2 L2 7Dw2 B' 7Uw2 B2 7Uw2 U 7Rw2 U' L2 7Fw2 L2 D2 F2 7Lw2 B2 D 7Lw2 B2 D 7Bw2 7Rw2 D 6Bw2 D' B U B2 6Bw2 R' 6Lw2 D' F2 6Dw2 F 6Dw2 R2 6Dw2 B' 6Rw2 F L2 F 6Uw2 R2 U' 6Rw2 B2 6Bw2 6Lw2 D' U B2 U 6Lw2 6Bw2 D 6Rw2 F2 6Rw2 B2 5Fw2 D' 5Uw2 5Fw2 R' U2 L' F' U2 5Bw2 F' 5Rw2 F 5Lw2 D2 5Dw2 L2 5Uw2 5Rw2 B' 5Uw2 B2 5Lw2 5Fw2 U 5Bw2 D 5Rw2 D 5Rw2 F2 U' 5Fw2 U 5Fw2 D' 5Bw2 U 4Rw2 F R' U B 4Rw2 B' L' R2 U' U2 F R2 F 4Dw2 4Uw2 F U2 B' 4Uw2 F 4Dw2 4Fw2 U2 4Rw2 U 4Bw2 F2 R2 4Fw2 4Rw2 D U2 F2 4Rw2 U' B2 4Rw2 R' 3Dw2 R 3Rw2 U' F 3Rw2 F' R' F' U' B 3Lw2 3Rw2 D2 3Uw2 F U2 L2 3Lw2 D2 F' 3Lw2 F' 3Fw2 D' 3Bw2 L2 3Rw2 D R2 3Rw2 3Fw2 U L2 3Bw2 D R2 3Fw2 2Bw2 D2 2Bw2 U 2Fw2 R D' 2Fw2 L' B D' D2 2Rw2 U2 F2 2Dw2 R2 2Rw2 B' 2Bw2 2Uw2 B' 2Lw2 B 2Bw2 2Dw2 U' 2Rw2 F2 2Rw2 2Fw2 U R2 B2 D2 U 2Rw2 2Bw2 L2 2Bw2 B U R L' B' R2 D' R D2 F' R B2 D' F2 D2 B2 R2 U L2 F2 D2"
mmoves = '.'.join([M[m] for m in sol.split(' ')])
len(mmoves.split("."))

84553

In [7]:
GREEDY_REDUCTION = True

outputs = {}
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(281, 284):
            continue
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = move_translation(dim)

        init_state = row['initial_state']
        sol_state = row['solution_state']

        # Only attempt to solve cubes with traditional solutions. 
        # That is, cubes that have a solution state where each face 
        # filled with equivilent letters/colors.
        if sol_state[:2*dim**2-1] == ';'.join(['A']*dim**2):
            state = state2ubl(init_state)
            print(f'Starting {id}')
            print(state)
            output = !./rubiks-cube-solver.py --state $state
            outputs[id] = output
            if output[-1][:9] == 'Solution:':
                sol = output[-1].split(': ')[1]
            else:
                for n in range(1, 21):
                    if 'Solution:' in output[-n]:
                        sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                        break
                
            mmoves = '.'.join([M[m] for m in sol.split(' ')])
            
            new_state = init_state
            for move in mmoves.split('.'):
                new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
            
            I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
            manipulations = [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for
                                                                              i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
            for init_moves in manipulations:
                temp_state = new_state
                if len(init_moves) > 0:
                    for move in init_moves.split('.'):
                        temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
                if temp_state == sol_state:
                    print(f'solved id: {id}')
                    if len(init_moves) > 0:
                        mmoves += '.' + init_moves
                    break
                    
            # validation
            state = row['initial_state'].split(";")
            for move_name in mmoves.split('.'):
                state = [state[i] for i in moves[move_name]]

            try:
                
                assert row['solution_state'].split(";") == state
                
                if GREEDY_REDUCTION:
                    # reduce moves
                    mmoves = solve_puzzle(puzzles.loc[id], mmoves)


                mmoves_length = len(mmoves.split('.'))
                best_moves_length = len(submission.loc[id, 'moves'].split('.')) 
                
                if mmoves_length < best_moves_length:
                    # Insert the moves into the database
                    submission.loc[id, 'moves'] = mmoves
            except AssertionError:
                print(f"assertion error for {id}")
                print(row['solution_state'])
                print(';'.join(state))
                break


Starting 281
FDRFLDUDDRFUDUBRRFLLRBFBLBRBFDRURUFUULLULUBRRRDRDULLULBBDBFDLBFBFBRLDUULBDLDRUURLFUDFLBLLDRFUFBUULBUFUBFFDDRBRULBRRDLLFUDRBDUURDFBBFBDBBBFDRRLFDBURDDLBDLUFFFFBLFUURUDBFRFRDRFLRRLBDBBFDULRUDFFRBBRDRRDBDBDLLFRDFDBUUDUFBULLRUBFFFFDUURURDLFURBBLFBBRLUFFDRBRRFFFLRDFDRDBBFFDBLRRLUBFDUUFDFLRDBRRRBFRURDBFUUDLLLBBUFRRUDUUULURUURBDFBBRDRDBRFBFRDDRDUFBUDFRRFDRLLDFBRUUUUDFRFRDUFDLRFUFUUFRBDDUBLDFDLDBDLRURDULLBUUBRDULULFUFLUBFDLRRURUFUUFRBURFDRRLBLRDDLURRLRRLRRBBLLLDRFUFFLDRBUUFUUDRUUBRUUBDLDBLFFDLFLULLLFUBULLFLUDURFFRFFFFFDDULDUDBBBDFDRBFBULFLFBBBRRBDURULFFLRRFDUFFLRLFULLULBBRFDUDRBUDLULDULLDUDBBRDLRFRURLFBLRDLLDULUBBDDUDLRFDBDLFURRURDBBRDFFUFFRDDFURULFFURBRRDDUFULBUDDBBLLDRRBDBDUBFLFBRRDFBLLLRBRBUBDFFBBRLUDRRULRLUBDULBULBRLLRUURUFDFBLLFUDDFLBLFRRRDDRFULLUFFLDDFRUFLLURLDLRUUFUBFUDDUBBFDFBRRULULDDDFBBRFUDRLBDFRLBDBBFRBRFBUBFUFUDLLUBURFUUBBUDFULBUURFULLLURULUUBFRBBURLRBRFDDLLFUBDLULRRBLDLRULUDBDBULLRDFLRDLDDRBRBFDLRUUBDUULDFDFBLFUBFFDUUDRDRFFUUFBLUDBBUUURFRLDFUUBBDRUUBLUDFBUURRRDUBRRUDU

IndexError: list index out of range

<a id="5.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SCORE<br><div>

In [ ]:
print(f"Score: {submission['moves'].str.split('.').apply(lambda x: len(x)).sum()}")

<a id="5.2"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SUBMISSION<br><div>

In [ ]:
submission.reset_index().to_csv('submission.csv', index=False)

os.chdir('..')
print("Current Working Directory: ", os.getcwd())

submission.reset_index().to_csv('submission.csv', index=False)

FYI: There are methods available to reduce some of these solutions further.